In [2]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import util.util_validation as ut_val

from torchvision import transforms, datasets
from tqdm import tqdm

from networks.resnet_big import model_dict
from util.util_logging import open_csv_file

seaborn.set_style("darkgrid")

In [3]:
models_dict = {"CE_baseline": ["./save/SupCE/animals10/SupCE_animals10_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_baseline_cosine/models/last.pth", None],
               "CE_origAllAug": ["./save/SupCE/animals10_diff_-1/SupCE_animals10_diff_-1_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_origAllAug_cosine/models/last.pth", None],
               "CE_diff4000": ["./save/SupCE/animals10_diff_4000/SupCE_animals10_diff_4000_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_4000_cosine/models/last.pth", None],
               "CE_diff4000AllAug": ["./save/SupCE/animals10_diff_4000/SupCE_animals10_diff_4000_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_4000AllAug_cosine/models/ckpt_epoch_250.pth", None],
               "CE_diff8000": ["./save/SupCE/animals10_diff_8000/SupCE_animals10_diff_8000_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_8000_cosine/models/last.pth", None],
               "CE_diffAug": ["./save/SupCE/animals10_diff_-1+4000/SupCE_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_diffAug_cosine/models/last.pth", None],
               "CE_diffAugAllAug": ["./save/SupCE/animals10_diff_-1+4000/SupCE_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_diffAugAllAug_cosine/models/last.pth", None],
               "SupCon_baseline": ["./save/SupCon/animals10_diff_-1/SupCon_animals10_diff_-1_resnet18_lr_0.125_decay_0.0001_bsz_26_temp_0.1_trial_0_try3_cosine/models/last.pth", ""],
               "SupCon_diff4000NoAug": ["./save/SupCon/animals10_diff_-1+4000/SupCon_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_temp_0.1_trial_0_noAug_cosine/models/last.pth", ""],
               "SupCon_diffCSameSAug": ["./save/SupCon/animals10_diff_-1+4000/SupCon_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_temp_0.1_trial_0_colorAugSameShapeAug_cosine/models/last.pth", ""],
               "SupCon_diffAllSameAug": ["./save/SupCon/animals10_diff_-1+4000/SupCon_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_temp_0.1_trial_0_allSameAug_cosine/models/ckpt_epoch_150.pth", ""],
               "SupConHybrid_diffNoAug": ["./save/SupCon/animals10_diff_-1+4000/SupConHybrid_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_temp_0.1_trial_0_noAug_cosine/models/last.pth", ""],
               "SupConHybrid_diffColorAug": ["./save/SupCon/animals10_diff_-1+4000/SupConHybrid_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_temp_0.1_trial_0_colorAug_cosine/models/last.pth", ""],
               "SupConFactor_factor5cAugSameSAug": ["./save/SupCon/animals10_diff_-1+4000/SupCon_5.0_animals10_diff_-1+4000_resnet18_lr_0.125_decay_0.0001_bsz_26_temp_0.1_trial_0_factor5cAugSameSAug_cosine/models/last.pth", ""],
               "CE_city": ["./save/SupCE/city_classification_original/SupCE_city_classification_original_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_cityBaseline_cosine/models/last.pth", None],
               "CE_cityDiff": ["./save/SupCE/city_classification_diff/SupCE_city_classification_diff_resnet18_lr_0.125_decay_0.0001_bsz_26_trial_0_cityDiff_cosine/models/last.pth", None]}

In [79]:
cuda_device = 0

model_short_name = "CE_baseline"
root_model = models_dict[model_short_name][0]

root_dataset = "./datasets/adaIN/shape_texture_conflict_animals10_many/"

path_folder, epoch = ut_val.get_paths_from_model_checkpoint(root_model)

params = open_csv_file(os.path.join(path_folder, "params.csv"))

## Set Dataloader and Model

In [4]:
normalize = transforms.Normalize(mean=params['mean'], std=params['std'])
val_transform = transforms.Compose([transforms.Resize(params['size']), transforms.CenterCrop(params['size']), transforms.ToTensor(), normalize])

conflict_dataset = ut_val.shapeTextureConflictDataset(root_dataset, val_transform)
classes = conflict_dataset.classes

conflict_dataloader = torch.utils.data.DataLoader(conflict_dataset, batch_size=params['batch_size'],
                                                  shuffle=False, num_workers=16, pin_memory=True)

model = ut_val.set_model(root_model, params, cuda_device)

## Compute Embedding

In [5]:
_, embedding_size = model_dict[params['model']]

model.eval()

embedding = np.array([])
shape_labels = np.array([], dtype=int)
texture_labels = np.array([], dtype=int)
with torch.no_grad():
    for images, labels in tqdm(conflict_dataloader):
        images = images.cuda(device=cuda_device, non_blocking=True)

        features = model.encoder(images)

        embedding = np.append(embedding, features.cpu().numpy())
        shape_labels = np.append(shape_labels, labels[0].numpy())
        texture_labels = np.append(texture_labels, labels[1].numpy())

embedding = embedding.reshape(-1, embedding_size)

100%|██████████| 897/897 [04:41<00:00,  3.18it/s]


## Estimate Dimension

In [6]:
shape_texture_name_list = [path.replace(".jpg", '').split('/')[-3:] for path in conflict_dataset.paths]
shapeName_textureName_list = [(s+'/'+n.split('_stylized_')[0], t+'/'+n.split('_stylized_')[1]) for s,t,n in shape_texture_name_list]

shape_pairs = []
shape_array = np.array([sN for sN,_ in shapeName_textureName_list])
for sN in set(shape_array):
    shape_indices = np.where(shape_array == sN)[0]
    shape_pairs.append(shape_indices)

texture_pairs = []
texture_array = np.array([tN for _,tN in shapeName_textureName_list])
for tN in set(texture_array):
    texture_indices = np.where(texture_array == tN)[0]
    texture_pairs.append(texture_indices)

shape_pair_A = np.concatenate([np.tile(shape_pairs[i], reps=len(shape_pairs[i])-1) for i in range(len(shape_pairs))])
shape_pair_B = np.concatenate([np.concatenate([np.roll(shape_pairs[i], shift=j) for j in range(1,len(shape_pairs[i]))]) for i in range(len(shape_pairs))])

texture_pair_A = np.concatenate([np.tile(texture_pairs[i], reps=len(texture_pairs[i])-1) for i in range(len(texture_pairs))])
texture_pair_B = np.concatenate([np.concatenate([np.roll(texture_pairs[i], shift=j) for j in range(1,len(texture_pairs[i]))]) for i in range(len(texture_pairs))])

In [7]:
def compute_correlation_score(embedding_A, embedding_B):
    A = torch.tensor(embedding_A)
    B = torch.tensor(embedding_B)

    A_dm = A - A.mean(dim=0)
    B_dm = B - B.mean(dim=0)

    correlation = (A_dm.T * B_dm.T).sum(dim=1) / ((A_dm.T * A_dm.T).sum(dim=1) * (B_dm.T * B_dm.T).sum(dim=1)).sqrt()
    correlation = torch.nan_to_num(correlation, nan=0.0)

    return correlation.mean().item()

def estimate_dims(correlation_scores, embedding_size):
    scores = np.array(np.concatenate((correlation_scores, [1.0])))

    m = np.max(scores)
    e = np.exp(scores-m)
    softmaxed = e / np.sum(e)

    dim = embedding_size
    dims = [int(s*dim) for s in softmaxed]
    dims[-1] = dim - sum(dims[:-1])

    return dims

In [8]:
corr_score_shape = compute_correlation_score(embedding[shape_pair_A], embedding[shape_pair_B])
corr_score_texture = compute_correlation_score(embedding[texture_pair_A], embedding[texture_pair_B])

dims = estimate_dims([corr_score_shape, corr_score_texture], embedding_size)
dims

[90, 182, 240]

epoch: last
- CE_baseline: [93, 179, 240]
- CE_origAllAug: [99, 169, 244]
- CE_diff4000: [89, 184, 239]
- CE_diff8000: [90, 182, 240]
- CE_diffAug: [90, 184, 238]
- CE_diffAugAllAug: [96, 173, 243]
- SupCon_baseline: [102, 159, 251]
- SupCon_diff4000NoAug: [90, 182, 240]
- SupCon_diffCSameSAug: [104, 158, 250]
- SupConHybrid_diffColorAug: [105, 161, 246]
- SupConFactor_factor5cAugSameSAug: [104, 157, 251]

epoch: 150
- SupConFactor_factor5cAugSameSAug: [100, 162, 250]

-------------

## Compute Embeddings for ShuffledPatches Dataset

In [5]:
import pickle
import torch.nn.functional as nnf

class ShufflePatches:
    # inspired from https://stackoverflow.com/questions/66962837/shuffle-patches-in-image-batch
    def __init__(self, patch_size):
        self.patch_size = patch_size

    def __call__(self, x):
        # unfold the tensor image
        u = nnf.unfold(x, kernel_size=self.patch_size , stride=self.patch_size , padding=0)
        # shuffle the patches in unfolded form
        pu = u[:,torch.randperm(u.shape[-1])]
        # fold the tensor back in its original form
        f = nnf.fold(pu, x.shape[-2:], kernel_size=self.patch_size, stride=self.patch_size, padding=0)

        return f

In [6]:
dataset_orig = "animals10_diff_-1"
dataset_diff = "animals10_diff_4000"
# dataset_orig = "city_classification_original"
# dataset_diff = "city_classification_diff"

_, path_embeddings_orig, path_embeddings_diff, _ = ut_val.get_paths_from_model_checkpoint(root_model, dataset_1=dataset_orig, dataset_2=dataset_diff)

_, root_dataset_orig = ut_val.get_root_dataset(dataset=dataset_orig)

normalize = transforms.Normalize(mean=params['mean'], std=params['std'])
shufflePatchers_transform = transforms.Compose([transforms.ToTensor(), ShufflePatches(patch_size=30), normalize])

shufflePatches_dataset = datasets.ImageFolder(root=root_dataset_orig,transform=shufflePatchers_transform)

shufflePatches_dataloader = torch.utils.data.DataLoader(shufflePatches_dataset, batch_size=params['batch_size'],
                                                        shuffle=False, num_workers=16, pin_memory=True)

model = ut_val.set_model(root_model, params, cuda_device)


# compute shuffled patches embedding
_, embedding_size = model_dict[params['model']]

model.eval()

embedding_test_patch = np.array([])
labels_test_patch = np.array([], dtype=int)
with torch.no_grad():
    for images, labels in tqdm(shufflePatches_dataloader):
        images = images.cuda(device=cuda_device, non_blocking=True)

        features = model.encoder(images)

        embedding_test_patch = np.append(embedding_test_patch, features.cpu().numpy())
        labels_test_patch = np.append(labels_test_patch, labels.numpy())

embedding_test_patch = embedding_test_patch.reshape(-1, embedding_size)


# compute original and diffused embeddings
with open(os.path.join(path_embeddings_orig, "embedding_test"), 'rb') as f:
    entry = pickle.load(f, encoding='latin1')
    embedding_test_orig = entry['data']
    class_labels_test = entry['labels']

with open(os.path.join(path_embeddings_diff, "embedding_test"), 'rb') as f:
    entry = pickle.load(f, encoding='latin1')
    embedding_test_diff = entry['data']

100%|██████████| 202/202 [01:03<00:00,  3.17it/s]


In [86]:
patchesColor_transform = transforms.Compose([transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
                                             transforms.ToTensor(), ShufflePatches(patch_size=30), normalize])

patchesColor_dataset = datasets.ImageFolder(root=root_dataset_orig,transform=patchesColor_transform)

patchesColor_dataloader = torch.utils.data.DataLoader(patchesColor_dataset, batch_size=params['batch_size'],
                                                        shuffle=False, num_workers=16, pin_memory=True)

embedding_test_patchesColor = np.array([])
with torch.no_grad():
    for images, labels in tqdm(patchesColor_dataloader):
        images = images.cuda(device=cuda_device, non_blocking=True)

        features = model.encoder(images)

        embedding_test_patchesColor = np.append(embedding_test_patchesColor, features.cpu().numpy())

embedding_test_patchesColor = embedding_test_patchesColor.reshape(-1, embedding_size)

100%|██████████| 202/202 [01:00<00:00,  3.32it/s]


In [80]:
_, root_dataset_diff = ut_val.get_root_dataset(dataset=dataset_diff)

color_transform = transforms.Compose([transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
                                      transforms.ToTensor(), normalize])

diff_dataset = datasets.ImageFolder(root=root_dataset_diff,transform=color_transform)

diff_dataloader = torch.utils.data.DataLoader(diff_dataset, batch_size=params['batch_size'],
                                                        shuffle=False, num_workers=16, pin_memory=True)

embedding_test_diffColorJitter = np.array([])
with torch.no_grad():
    for images, labels in tqdm(diff_dataloader):
        images = images.cuda(device=cuda_device, non_blocking=True)

        features = model.encoder(images)

        embedding_test_diffColorJitter = np.append(embedding_test_diffColorJitter, features.cpu().numpy())

embedding_test_diffColorJitter = embedding_test_diffColorJitter.reshape(-1, embedding_size)

100%|██████████| 202/202 [01:01<00:00,  3.31it/s]


## Estimate Dimension for ShuffledPatches Dataset

In [87]:
def compute_correlation_score(embedding_A, embedding_B):
    A = torch.tensor(embedding_A)
    B = torch.tensor(embedding_B)

    A_dm = A - A.mean(dim=0)
    B_dm = B - B.mean(dim=0)

    correlation = (A_dm * B_dm).sum(dim=0) / ((A_dm * A_dm).sum(dim=0) * (B_dm * B_dm).sum(dim=0)).sqrt()
    correlation = torch.nan_to_num(correlation, nan=0.0)

    return correlation.mean().item()

def estimate_dims(correlation_scores, embedding_size):
    scores = np.array(np.concatenate((correlation_scores, [1.0])))

    m = np.max(scores)
    e = np.exp(scores-m)
    softmaxed = e / np.sum(e)

    dim = embedding_size
    dims = [int(s*dim) for s in softmaxed]
    dims[-1] = dim - sum(dims[:-1])

    return dims

dim_estimate_dict = dict()

In [88]:
corr_score_shape = compute_correlation_score(embedding_test_orig, embedding_test_diffColorJitter)
corr_score_texture = compute_correlation_score(embedding_test_orig, embedding_test_patchesColor)
corr_score_color = compute_correlation_score(embedding_test_diff, embedding_test_patch)

dims = estimate_dims([corr_score_shape, corr_score_texture, corr_score_color], embedding_size)

print(f"shape: {dims[0]:>3}, texture: {dims[1]:>3}, color: {dims[2]:>3}, other: {dims[3]:>3}")

shape: 135, texture:  91, color: 114, other: 172


In [83]:
corr_score_shape = compute_correlation_score(embedding_test_orig, embedding_test_diffColorJitter)
corr_score_texture = compute_correlation_score(embedding_test_orig, embedding_test_patch)

dims = estimate_dims([corr_score_shape, corr_score_texture], embedding_size)

print(f"shape: {dims[0]:>3}, texture: {dims[1]:>3}, other: {dims[2]:>3}")

shape: 163, texture: 141, other: 208


In [98]:
corr_score_shape = compute_correlation_score(embedding_test_orig, embedding_test_diff)
corr_score_texture = compute_correlation_score(embedding_test_orig, embedding_test_patch)
corr_score_color = compute_correlation_score(embedding_test_diff, embedding_test_patch)

dims = estimate_dims([corr_score_shape, corr_score_texture, corr_score_color], embedding_size)

print(f"shape: {dims[0]:>3}, texture: {dims[1]:>3}, color: {dims[2]:>3}, other: {dims[3]:>3}")

dim_estimate_dict[model_short_name] = dims

shape: 131, texture: 113, color: 100, other: 168


In [99]:
df_est_dims = pd.DataFrame.from_dict(dim_estimate_dict, orient="index", columns=["shape dims", "texture dims", "color dims", "other dims"])
df_est_dims

,shape dims,texture dims,color dims,other dims
CE_city,122,115,101,174
CE_cityDiff,131,113,100,168


In [100]:
print(df_est_dims.sort_values("shape dims", ascending=False).to_markdown())

|             |   shape dims |   texture dims |   color dims |   other dims |
|:------------|-------------:|---------------:|-------------:|-------------:|
| CE_cityDiff |          131 |            113 |          100 |          168 |
| CE_city     |          122 |            115 |          101 |          174 |


|                                  |   shape dims |   texture dims |   color dims |   other dims |
|:---------------------------------|-------------:|---------------:|-------------:|-------------:|
| SupConFactor_factor5cAugSameSAug |          165 |             87 |           87 |          173 |
| SupConHybrid_diffColorAug        |          163 |             88 |           88 |          173 |
| SupCon_diffCSameSAug             |          157 |             93 |           93 |          169 |
| CE_diffAugAllAug                 |          156 |             94 |           95 |          167 |
| SupCon_diff4000NoAug             |          153 |             99 |           98 |          162 |
| CE_diffAug                       |          149 |            102 |          103 |          158 |
| CE_diff4000                      |          135 |            110 |           99 |          168 |
| CE_origAllAug                    |          133 |            103 |          103 |          173 |
| SupConHybrid_diffNoAug           |          132 |            120 |          120 |          140 |
| CE_diff8000                      |          131 |            112 |           98 |          171 |
| CE_baseline                      |          130 |            110 |          109 |          163 |
| SupCon_baseline                  |          129 |            106 |          100 |          177 |

|             |   shape dims |   texture dims |   color dims |   other dims |
|:------------|-------------:|---------------:|-------------:|-------------:|
| CE_cityDiff |          131 |            113 |          100 |          168 |
| CE_city     |          122 |            115 |          101 |          174 |